# Call cellular infection status and viral tag assignment for infected cells
This Python Jupyter notebook uses the viral tags to determine which cells were infected, and the tag variant of the infecting virus for infected cells.

Import Python modules:

In [ ]:
import textwrap

import anndata

from dms_variants.constants import CBPALETTE

import kneed

import matplotlib.pyplot as plt

import numpy

import pandas as pd

import scanpy

import scipy

import statsmodels.stats.multitest

Get `snakemake` variables [as described here](https://snakemake.readthedocs.io/en/stable/snakefiles/rules.html#jupyter-notebook-integration):

In [ ]:
viral_tag_by_cell_csv = snakemake.input.viral_tag_by_cell_csv
matrix = snakemake.input.matrix
features = snakemake.input.features
cell_barcodes = snakemake.input.cell_barcodes
cell_annotations = snakemake.output.cell_annotations
plot = snakemake.output.plot
viral_genes = snakemake.params.viral_genes
viral_tags = snakemake.params.viral_tags
fdr = snakemake.params.fdr
expt = snakemake.wildcards.expt

This notebook assumes that there are exactly two viral tags, code does not work otherwise:

In [ ]:
if len(viral_tags) != 2:
    raise ValueError('code assumes exactly two viral tags')

Read the cell-gene matrix into an [AnnData](https://anndata.readthedocs.io/) object:

In [ ]:
adata = scanpy.read_mtx(matrix)
adata.var = pd.read_csv(cell_barcodes,
                        names=['cell_barcode'])
adata.obs = pd.read_csv(features,
                        sep='\t',
                        names=['ensemble_id', 'gene', 'feature_type'])

print(f"Read cell-gene matrix of {adata.n_vars} cells and {adata.n_obs} genes")

assert set(viral_genes).issubset(set(adata.obs['gene'])), 'cell-gene matrix missing viral genes'

Now get for each cell:
 - total UMI corrected reads
 - UMI corrected viral and cellular
 - fraction of UMIs that are viral and cellular

In [ ]:
umi_counts = (
    adata.var
    .assign(total_UMIs=numpy.sum(adata.X, axis=0).A1.astype(int),
            viral_UMIs=numpy.sum(adata[adata.obs['gene'].isin(viral_genes),].X,
                                 axis=0).A1.astype(int),
            cellular_UMIs=numpy.sum(adata[~adata.obs['gene'].isin(viral_genes),].X,
                                    axis=0).A1.astype(int),
            frac_viral_UMIs=lambda x: x['viral_UMIs'] / x['total_UMIs'],
            frac_cellular_UMIs=lambda x: x['cellular_UMIs'] / x['total_UMIs'],
            )
    )

Now we make some plots showing the amount of total and viral UMIs per cell.
We are going to eventually make a merged plot here, so we create a multi-axis subplots figure and then just start adding plots to it as we go through this notebook:

In [ ]:
# set up master figure subplots that we then add to throughout notebook
ncols = 4
nrows = 4
fig, axes = plt.subplots(ncols=ncols,
                         nrows=nrows,
                         figsize=(3.5 * ncols, 3 * nrows)
                         )
i_axis = 0

# distribution of total and viral UMIs per cell
for umi_type, ax in zip(['total', 'viral'], axes.ravel()):
    _ = umi_counts.plot(y=f"{umi_type}_UMIs",
                        kind='hist',
                        bins=25,
                        ax=ax,
                        legend=False,
                        color='black',
                        )
    _ = ax.set_xlabel(f"{umi_type} UMIs per cell")
    _ = ax.set_ylabel('number of cells')
    _ = ax.set_title(f"{umi_type} UMIs per cell")
    i_axis += 1
    
# fraction of UMIs that are viral
ax = umi_counts.plot(y='frac_viral_UMIs',
                     kind='hist',
                     bins=25,
                     ax=axes.ravel()[i_axis],
                     legend=False,
                     color='black',
                     )
_ = ax.set_xlabel('fraction of UMIs from virus')
_ = ax.set_ylabel('number of cells')
_ = ax.set_title('fraction of UMIs from virus')
i_axis += 1

# correlation of viral and cellular UMIs
ax = umi_counts.plot(x='cellular_UMIs',
                     y='viral_UMIs',
                     kind='scatter',
                     ax=axes.ravel()[i_axis],
                     alpha=0.1,
                     legend=False,
                     color='black',
                     )
_ = ax.set_xlabel('cellular UMIs per cell')
_ = ax.set_ylabel('viral UMIs per cell')
_ = ax.set_title('viral vs cellular UMIs per cell')
i_axis += 1

# show figure
fig.tight_layout()

Combine UMI counts with counts of viral tags, indentifying the "major" (more abundant) and "minor" (less abundant) viral tag for each cell, and also computing their ratio to total UMIs in each cell.

In [ ]:
# read tag counts in tidy form
tidy_tag_counts = pd.read_csv(viral_tag_by_cell_csv)

# check tag count values
assert set(tidy_tag_counts['tag_variant']).issubset(set(viral_tags)), 'unrecognized viral tag'
assert set(tidy_tag_counts['cell_barcode']).issubset(set(umi_counts['cell_barcode'])), 'unrecognized cell barcode'
assert set(tidy_tag_counts['gene']).issubset(set(viral_genes)), 'unrecognized viral gene'

# for each cell barcode, identify more abundant (major) and less abundant (minor) tag and get counts
tag_counts = (
    tidy_tag_counts
    # sum counts across viral genes
    .groupby(['cell_barcode', 'tag_variant'], as_index=False)
    .aggregate(count=pd.NamedAgg('count', 'sum'))
    # fill counts of 0 for cells missing any counts for a tag
    .merge(pd.concat([pd.DataFrame({'cell_barcode': umi_counts['cell_barcode'],
                         'tag_variant': viral_tag})
                      for viral_tag in viral_tags]),
           how='right', on=['cell_barcode', 'tag_variant'])
    .assign(count=lambda x: x['count'].fillna(0).astype(int))
    # get major and minor viral tag and their counts
    .sort_values('count', ascending=False)
    .groupby('cell_barcode', as_index=False)
    .aggregate(viral_tag_major=pd.NamedAgg('tag_variant', 'first'),
               viral_tag_major_counts=pd.NamedAgg('count', 'max'),
               viral_tag_minor=pd.NamedAgg('tag_variant', 'last'),
               viral_tag_minor_counts=pd.NamedAgg('count', 'min')
               )
    # add the UMI counts
    .merge(umi_counts, on='cell_barcode', how='outer')
    # compute ratio of viral tag to total UMIs
    .assign(viral_tag_major_to_total_UMI=lambda x: x['viral_tag_major_counts'] / x['total_UMIs'],
            viral_tag_minor_to_total_UMI=lambda x: x['viral_tag_minor_counts'] / x['total_UMIs'])
    )

assert tag_counts.notnull().all().all()

Plot the ratio of minor and major tags to UMIs for each cell, and also the distribution of fraction of UMIs from virus for cells stratified by which viral tag variant is the major one:

In [ ]:
# correlation of minor and major tag counts to total UMI ratios
ax = tag_counts.plot(x='viral_tag_major_to_total_UMI',
                     y='viral_tag_minor_to_total_UMI',
                     kind='scatter',
                     ax=axes.ravel()[i_axis],
                     alpha=0.1,
                     c='black',
                     legend=False,
                     )
_ = ax.set_xlabel('major tag counts / total UMIs')
_ = ax.set_ylabel('minor tag counts / total UMIs')
_ = ax.set_title('major and minor viral tags per cell')
i_axis += 1

# fraction of UMIs that are viral stratified by which is major tag variant
tag_to_color = dict(zip(viral_tags, CBPALETTE[1:]))
for tag in viral_tags:
    ax = tag_counts.query('viral_tag_major == @tag').plot(
                    y='frac_viral_UMIs',
                    kind='hist',
                    bins=25,
                    alpha=0.5,
                    color=tag_to_color[tag],
                    ax=axes.ravel()[i_axis],
                    legend=False,
                    )
ax.legend(labels=viral_tags, title='major viral tag')
_ = ax.set_xlabel('fraction of UMIs from virus')
_ = ax.set_ylabel('number of cells')
_ = ax.set_title('fraction UMIs from virus by major tag')
i_axis += 1

# display figure
fig.tight_layout()
display(fig)

Now we identify the cells that are clearly **not** infected with each viral tag variant.
The logic is as follows: the cells were independently infected with the two tag variants, and the only mixed right before sequencing.
So even in infected cells, the "minor" tag variant should always be at the background level expected of uninfected cells.
The only exception is for doublets of cells infected with the two tag variants.
Therefore, we divide cells by which tag variant is the "minor" tag, and then make a knee plot showing the cell rank versus the counts of that tag normalized by total UMIs.
This should be a near flat line except for an upward "knee" near the end for doublets of cells infected with the two tag variants.
We use the [kneed](https://kneed.readthedocs.io/) package to identify this knee, and  then plot the knee plots.

In [ ]:
# identify the "knee" for each minor viral tag variant
knee = {}
knee_y = {}
for tag, ax in zip(viral_tags, axes.ravel()[i_axis: ]):
    
    # identify knee
    df = (tag_counts
          .query('viral_tag_minor == @tag')
          .sort_values('viral_tag_minor_to_total_UMI')
          .assign(cell_rank=lambda x: x['viral_tag_minor_to_total_UMI'].rank(method='first'))
          )
    kl = kneed.KneeLocator(x=df['cell_rank'].tolist(),
                           y=df['viral_tag_minor_to_total_UMI'].tolist(),
                           curve='convex',
                           direction='increasing',
                           S=3,
                           )
    knee[tag] = kl.knee
    knee_y[tag] = kl.knee_y
    
    # plot cells above / below knee
    ax = df.assign(color=lambda x: (x['viral_tag_minor_to_total_UMI'] < knee_y[tag])
                                    .map({True: CBPALETTE[3], False: CBPALETTE[4]})
                   ).plot(
                x='cell_rank',
                y='viral_tag_minor_to_total_UMI',
                kind='scatter',
                s=1.5,
                c='color',
                ax=ax)
    _ = ax.set_xlabel('cell rank')
    _ = ax.set_ylabel('minor tag / total UMIs')
    n_below = (df['viral_tag_minor_to_total_UMI'] < knee_y[tag]).sum()
    n_above = len(df) - n_below
    _ = ax.set_title(f"{tag} minor tag knee plot:\n{n_below} below & {n_above} above knee")
    i_axis += 1
    ax.axvline(x=knee[tag], color='black', alpha=0.5, linestyle='--')
    
# display figure
fig.tight_layout()
display(fig)
   
# annotate tag counts by whether below knee (uninfected with that tag)
tag_counts = (
    tag_counts
    .assign(viral_tag_minor_below_knee=lambda x: x.apply(lambda r: (r['viral_tag_minor_to_total_UMI'] <
                                                                    knee_y[r['viral_tag_minor']]),
                                                         axis=1))
    )

Now look at the 95th percentile of tag counts to total UMIs for each tag using cells that are below the knee and for which that tag is the minor tag.
This enables us to establish a reasonable upper bound on the **expected** number of observations of that viral tag in uninfected cells based on the mean of the ratio and the total UMIs in that cell.
Note that the 95th percentile doesn't mean that 5% of cells are called infected, because we later require significantly **more** viral tag counts than this number, but it gives some sense of what we expect to be upper amount before looking at statistical noise in tag counts per cell:

In [ ]:
minor_tag_to_total_UMI = {}
for tag, ax in zip(viral_tags, axes.ravel()[i_axis: ]):
    df = (tag_counts
          .query('viral_tag_minor == @tag')
          .query('viral_tag_minor_below_knee')
          )
    minor_tag_to_total_UMI[tag] = df['viral_tag_minor_to_total_UMI'].quantile(0.95)
    df.plot(y='viral_tag_minor_to_total_UMI',
            kind='hist',
            bins=25,
            color=tag_to_color[tag],
            ax=ax,
            legend=None,
            )
    _ = ax.set_xlabel('minor tag counts / total UMIs')
    _ = ax.set_ylabel('number of cells')
    _ = ax.set_title(f"{tag} minor tag in cells below knee")
    ax.axvline(x=minor_tag_to_total_UMI[tag],
               color='black', linestyle='--')
    ax.annotate(f"{minor_tag_to_total_UMI[tag]:.2g}",
                xy=(minor_tag_to_total_UMI[tag] * 0.98, 0.95 * ax.get_ylim()[1]),
                ha='right',
                va='top',
                )
    i_axis += 1
    
# display figure
fig.tight_layout()
display(fig)

Now directly compute the **expected** number of observations of each viral tag if cell is uninfected based on the mean minor tag to total UMI for that tag in cells below the knee.
Then use this to calculate a P-value (assuming Poisson statistics) for each cell being infected with each viral tag variant, as well as a Q-value and the infection status at the indicated false-discovery rate.
We then plot the P and Q values distribution for cells being infected by major and minor tag.
Finally, we plot whether cells that were below or above the "knee" on the minor tag : UMIs plot are called as infected by the minor tag.
If things are working, cells below the knee should not be called as infected by the minor viral tag, but most cells above knee should.

In [ ]:
print(f"Computing infection status at a FDR of <{fdr}")

# compute P and Q values for cells being infected with minor and major tag
for tag_type in ['minor', 'major']:
    
    # compute P values
    tag_counts[f"expected_{tag_type}_tag"] = (
            tag_counts['total_UMIs'] *
            numpy.array([minor_tag_to_total_UMI[itag] for itag in tag_counts[f"viral_tag_{tag_type}"]])
            )
    tag_counts[f"viral_tag_{tag_type}_infected_P"] = 1 - scipy.stats.poisson.cdf(
                                                tag_counts[f"viral_tag_{tag_type}_counts"],
                                                tag_counts[f"expected_{tag_type}_tag"])
    tag_counts[f"viral_tag_{tag_type}_infected_Q"] = statsmodels.stats.multitest.fdrcorrection(
                    pvals=tag_counts[f"viral_tag_{tag_type}_infected_P"],
                    alpha=fdr,
                    method='indep',
                    is_sorted=False)[1]
    tag_counts[f"viral_tag_{tag_type}_infected"] = tag_counts[f"viral_tag_{tag_type}_infected_Q"] < fdr

    # plot histograms of P- and Q-values
    for stat, color in zip(['P', 'Q'], CBPALETTE[3: ]):
        ax = tag_counts.plot(
                    y=f"viral_tag_{tag_type}_infected_{stat}",
                    kind='hist',
                    bins=40,
                    alpha=0.5,
                    color=color,
                    ax=axes.ravel()[i_axis],
                    legend=False,
                    )
    ax.legend(labels=['P-value', 'Q-value'])
    _ = ax.set_xlabel('P/Q-value rejecting uninfected')
    _ = ax.set_ylabel('number of cells')
    _ = ax.set_title(f"P/Q infected by {tag_type} viral tag")
    i_axis += 1

def label_bar_counts(bar_ax):
    """Label bars with counts: https://matplotlib.org/examples/api/barchart_demo.html"""
    for rect in bar_ax.patches:
        bar_ax.text(
            rect.get_x() + rect.get_width() / 2,
            rect.get_height(),
            f"{rect.get_height():d}",
            ha='center',
            va='bottom',
            color=rect.get_facecolor())
        
# now plot whether cells are infected by minor tag based on knee plot location
# plot number of cells below and above the "knee" called as infected by minor tag
ax = (
    tag_counts
    .groupby(['viral_tag_minor_below_knee', 'viral_tag_minor_infected'])
    .aggregate(ncells=pd.NamedAgg('cell_barcode', 'count'))
    .reset_index()
    .assign(viral_tag_minor_below_knee=lambda x: x['viral_tag_minor_below_knee'].map({True: 'below knee',
                                                                                      False: 'above knee'}),
            viral_tag_minor_infected=lambda x: x['viral_tag_minor_infected'].map({True: 'infected',
                                                                                        False: 'not infected'}),
            )
    .pivot_table(index='viral_tag_minor_below_knee',
                 columns='viral_tag_minor_infected',
                 values='ncells',
                 fill_value=0)
    .rename_axis(None, axis=1)
    .rename_axis(None)
    .plot(kind='bar', ax=axes.ravel()[i_axis])
    )
_ = ax.legend(loc='upper left')
_ = ax.set_ylabel('number of cells')
_ = ax.set_title('minor tag infection by knee plot')
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
_ = ax.set_ylim((0, ax.get_ylim()[1] * 1.1))
label_bar_counts(ax)
i_axis += 1
    
# display figure
fig.tight_layout()
display(fig)

Now identify infected cells and indicate which viral tag variant infected them, also noting which cells are "doublets" from the viral tag information:

In [ ]:
def classify_infecting_viral_tag(r):
    if r['viral_tag_major_infected'] and r['viral_tag_minor_infected']:
        return 'both'
    elif r['viral_tag_major_infected']:
        return r['viral_tag_major']
    elif r['viral_tag_minor_infected']:
        return r['viral_tag_minor']
    else:
        return 'none'

tag_counts = (
    tag_counts
    .assign(
        infected=lambda x: x['viral_tag_major_infected'] | x['viral_tag_minor_infected'],
        viral_tag_doublet=lambda x: x['viral_tag_major_infected'] & x['viral_tag_minor_infected'],
        infecting_viral_tag=lambda x: pd.Categorical(x.apply(classify_infecting_viral_tag, axis=1),
                                                     [*viral_tags, 'both', 'none'],
                                                     ordered=True)
        )
    )

Plot number of cells infected with each viral tag variant, and distribution of how much viral RNA per cell (fraction of UMIs) on both linear and log scale:

In [ ]:
# bar plots of number of cells infected with each tag (or not infected or both tags)
ax = (
    tag_counts
    .groupby('infecting_viral_tag', as_index=False)
    .aggregate(n_cells=pd.NamedAgg('cell_barcode', 'count'))
    .assign(color=lambda x: x['infecting_viral_tag'].map(tag_to_color))
    .plot(x='infecting_viral_tag',
          y='n_cells',
          kind='bar',
          legend=None,
          ax = axes.ravel()[i_axis],
          color='black',
          )
    )
_ = ax.set_ylabel('number of cells')
_ = ax.set_xlabel('infecting viral tag')
_ = ax.set_title('number infected cells by viral tag')
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
_ = ax.set_ylim((0, ax.get_ylim()[1] * 1.1))
label_bar_counts(ax)
i_axis += 1

# box plots of amount of viral RNA per cell for each infecting tag category
for yscale in ['linear', 'log']:
    ax = tag_counts.boxplot(
        by='infecting_viral_tag',
        column='frac_viral_UMIs',
        boxprops={'color': 'black', 'linewidth': 1.5},
        whiskerprops={'color': 'black', 'linewidth': 1.5},
        medianprops={'color': 'black', 'linewidth': 1.5},
        flierprops={'alpha': 0.1,
                    'markerfacecolor': 'black',
                    'markeredgecolor': 'black',
                    'markersize': 3},
        ax=axes.ravel()[i_axis],
        )
    _ = ax.set_xlabel('infecting viral tag')
    _ = ax.set_ylabel('fraction UMIs from virus')
    _ = ax.set_title('viral load / cell by viral tag')
    _ = ax.grid(None)
    _ = ax.set_yscale(yscale)
    i_axis += 1

# display figure
fig.tight_layout()
display(fig)

Add title to plot and then save:

In [ ]:
fig.suptitle(f"Calling viral tags and infection status for {expt}", size=16)

print(f"Saving plots to {plot}")
fig.savefig(plot)

fig.tight_layout()
display(fig)

Get data frame with just columns of interest, display the first few lines, and then write to a file as a gzipped CSV:

In [ ]:
tag_counts_curated = (
    tag_counts
    [['cell_barcode', 'total_UMIs', 'viral_UMIs', 'cellular_UMIs', 'frac_viral_UMIs',
      'infected', 'infecting_viral_tag', 'viral_tag_doublet',
      'viral_tag_major', 'viral_tag_minor', 'viral_tag_major_infected', 'viral_tag_minor_infected',
      'viral_tag_major_counts', 'viral_tag_minor_counts']]
    )

display(tag_counts_curated)

assert (tag_counts_curated['cell_barcode'] == adata.var['cell_barcode']).all(), 'cell barcodes out of order'

print(f"Saving viral-tag annotated cell barcodes to {cell_annotations}")
tag_counts_curated.to_csv(cell_annotations,
                          compression='gzip',
                          index=False)